In [1]:
# Dependencies
from pprint import pprint
import pymongo
import tweepy
import json
import numpy as np
import spacy
nlp = spacy.load('en')


#connection with Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.project_reax
db_cities_full = db.cities_full

In [2]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [32]:
i = 0
trend_woeid = 23424977

for trend in db.twitter_trends_full.find({"locations.woeid": trend_woeid})[0]["trends"]:
    i = i+1
    if (i==6):
        break
        
    trend_name = trend["name"]

    words = ""
        
    for key_set in trend["spacy_npl"]["nlp_key"]:
        for key in key_set:
            if (key != "no key" and "http" not in key):
                words = words + " " + key.replace("#","")
            
    keys_freq = []
    for word in words.split():
        found=False
        for key_pair in keys_freq:
            if (key_pair[0]==word):
                key_pair[1] = key_pair[1] + 1
                found=True
        if (not found):
            keys_freq.append([word,1])
            
    def takeSecond(elem):
        return elem[1]
    keys_freq.sort(key=takeSecond,reverse=True)
    keys_freq_dict =[]
    for key_pair in keys_freq:
        keys_freq_dict.append({"key": key_pair[0],"value": key_pair[1]})
        
    db.twitter_trends_full.update_one(
                {"locations.woeid": trend_woeid, "trends.name": trend_name},
                { "$set":
                    {"trends.$[trend].keys": keys_freq_dict}
                }
                ,array_filters=[{"trend.name": trend_name}]
                )    

In [35]:
db.twitter_trends_full.find({"locations.woeid": trend_woeid})[0]["trends"][4]["keys"]

[{'key': 'Vikings', 'value': 277},
 {'key': 'the', 'value': 53},
 {'key': 'Cousins', 'value': 43},
 {'key': 'Kirk', 'value': 41},
 {'key': '-', 'value': 30},
 {'key': '2', 'value': 24},
 {'key': 'Bears', 'value': 23},
 {'key': 'last', 'value': 23},
 {'key': 'NFC', 'value': 22},
 {'key': 'NFL', 'value': 22},
 {'key': 'Minnesota', 'value': 21},
 {'key': 'Sunday', 'value': 20},
 {'key': 'North', 'value': 19},
 {'key': 'Skol', 'value': 18},
 {'key': 'week', 'value': 18},
 {'key': 'one', 'value': 18},
 {'key': 'DaBears', 'value': 17},
 {'key': '️', 'value': 15},
 {'key': '3', 'value': 15},
 {'key': 'Saints', 'value': 15},
 {'key': 'The', 'value': 15},
 {'key': 'SNF', 'value': 15},
 {'key': 'Super', 'value': 15},
 {'key': 'Bowl', 'value': 15},
 {'key': '25-20', 'value': 15},
 {'key': 'Packers', 'value': 14},
 {'key': 'season', 'value': 14},
 {'key': '1', 'value': 14},
 {'key': '5', 'value': 14},
 {'key': '2018', 'value': 14},
 {'key': 'night', 'value': 14},
 {'key': 'year', 'value': 14},
 {'

In [8]:
i = 0
trend_woeid = 23424977

for trend in db.twitter_trends_full.find({"locations.woeid": trend_woeid})[0]["trends"]:
    i = i+1
    if (i==6):
        break
    
    trend_name = trend["name"]
    
    vader_scores_list = []

    for public_tweet in trend["public_tweets"]:
        if len(public_tweet["vader_scores"]) != 0:
            avg_vader = sum(public_tweet["vader_scores"]) / len(public_tweet["vader_scores"])
            vader_scores_list.append({"lat": public_tweet["latitude"], "lng": public_tweet["longitude"], "state": public_tweet["state"], "city": public_tweet["city"], "avg_vader": avg_vader})
        
    db.twitter_trends_full.update_one(
            {"locations.woeid": trend_woeid, "trends.name": trend_name},
            { "$set":
                {"trends.$[trend].vader_statistics": vader_scores_list}
            }
            ,array_filters=[{"trend.name": trend_name}]
            )   
        

In [9]:
db.twitter_trends_full.find({"locations.woeid": trend_woeid})[0]["trends"][4]["vader_statistics"]

[{'lat': 40.7127837,
  'lng': -74.0059413,
  'state': 'New York',
  'city': 'New York',
  'avg_vader': 0.24621999999999997},
 {'lat': 34.0522342,
  'lng': -118.2436849,
  'state': 'California',
  'city': 'Los Angeles',
  'avg_vader': -0.15551000000000004},
 {'lat': 41.8781136,
  'lng': -87.6297982,
  'state': 'Illinois',
  'city': 'Chicago',
  'avg_vader': 0.44752222222222215},
 {'lat': 29.7604267,
  'lng': -95.3698028,
  'state': 'Texas',
  'city': 'Houston',
  'avg_vader': -0.19717},
 {'lat': 39.9525839,
  'lng': -75.1652215,
  'state': 'Pennsylvania',
  'city': 'Philadelphia',
  'avg_vader': 0.04146},
 {'lat': 33.4483771,
  'lng': -112.0740373,
  'state': 'Arizona',
  'city': 'Phoenix',
  'avg_vader': 0.40962},
 {'lat': 29.4241219,
  'lng': -98.49362819999999,
  'state': 'Texas',
  'city': 'San Antonio',
  'avg_vader': 0.19922},
 {'lat': 32.715738,
  'lng': -117.1610838,
  'state': 'California',
  'city': 'San Diego',
  'avg_vader': 0.48685999999999996},
 {'lat': 32.7766642,
  'lng'